In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

In [ ]:
# all events
# Source: https://api.developer.attomdata.com/docs#!/All32Events32Extended32by32Address/getAllEventsDetailAddress
#https://api.gateway.attomdata.com/propertyapi/v1.0.0/allevents/detail?address1=4529%20Winona%20Court&address2=Denver%2C%20CO%2080212
    
    

In [19]:
def call_api_all(bld_num,street,str_type,unit=None,unit_num=None,city=None,state=None):
    import http.client
    
    conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 

    headers = { 
        'accept': "application/json", 
        'apikey': "keyID", 
    } 
    address_1 = f"address1={bld_num}%20{street}%20{str_type}%20{unit}%20{unit_num}"
    address_2 = f"&address2={city}%2C%20{state}"
    conn.request("GET", f"/propertyapi/v1.0.0/allevents/detail?{address_1}{address_2}", headers=headers) 

    res = conn.getresponse() 
    data = res.read() 
    
#    print(data.decode("utf-8"))
    print(address_1,address_2)
    return data
#    print("GET", f"/propertyapi/v1.0.0/property/detail?{address}", "headers=headers")

In [67]:

def extract_details(api_data):
    house = json.loads(api_data)
    
    # create new dfs based on dictionary keys

    # df_1 ['obPropId', 'fips', 'apn', 'apnOrig', 'attomId']
    df_1 = pd.DataFrame(house['property'][0]['identifier'],index=[0])

    # df_2 ['depth', 'frontage', 'lotnum', 'lotsize1', 'lotsize2', 'pooltype']
    df_2 = pd.DataFrame(house['property'][0]['lot'],index=[0])

    # df_3 ['blockNum', 'countrysecsubd', 'countyuse1', 'muncode', 'munname',
    #       'srvyRange', 'srvySection', 'srvyTownship', 'subdname', 'subdtractnum','taxcodearea']
    df_3 = pd.DataFrame(house['property'][0]['area'],index=[0])

    # df_4 ['country', 'countrySubd', 'line1', 'line2', 'locality', 'matchCode',
    #       'oneLine', 'postal1', 'postal2', 'postal3']
    df_4 = pd.DataFrame(house['property'][0]['address'],index=[0])

    # df_5 = ['accuracy', 'elevation', 'latitude', 'longitude', 'distance', 'geoid']
    df_5 = pd.DataFrame(house['property'][0]['location'],index=[0])

    # df_6 ['absenteeInd', 'propclass', 'propsubtype', 'proptype', 'yearbuilt',
    #       'propLandUse', 'propIndicator', 'legal1']
    df_6 = pd.DataFrame(house['property'][0]['summary'],index=[0])

    #df_7 ['heatingtype']
    df_7 = pd.DataFrame(house['property'][0]['utilities'],index=[0])

    #df_8 **['size', 'rooms', 'interior', 'construction', 'parking', 'summary']** dict keys
    df_8 = pd.DataFrame(house['property'][0]['building'],index=[0])

    #df_9 ['lastModified', 'pubDate']
    df_9 = pd.DataFrame(house['property'][0]['vintage'],index=[0])
    
    # create new dfs based on df_8 dictionary keys

    # df_b1 ['bldgsize', 'grosssize', 'grosssizeadjusted', 'groundfloorsize',
    #       'livingsize', 'sizeInd', 'universalsize']
    df_b1 = pd.DataFrame(house['property'][0]['building']['size'],index=[0])

    # df_b2 ['bathfixtures', 'baths1qtr', 'baths3qtr', 'bathscalc', 'bathsfull',
    #       'bathshalf', 'bathstotal', 'beds', 'roomsTotal']
    df_b2 = pd.DataFrame(house['property'][0]['building']['rooms'],index=[0])

    #df_b3 ['bsmtsize', 'bsmttype', 'fplccount', 'fplcind', 'fplctype']
    df_b3 = pd.DataFrame(house['property'][0]['building']['interior'],index=[0])

    #df_b4 ['condition', 'wallType']
    df_b4 = pd.DataFrame(house['property'][0]['building']['construction'],index=[0])

    #df_b5 ['garagetype', 'prkgSize', 'prkgSpaces', 'prkgType']
    df_b5 = pd.DataFrame(house['property'][0]['building']['parking'],index=[0])

    #df_b6 ['bldgsNum', 'bldgType', 'imprType', 'levels', 'mobileHomeInd',
    #       'quality', 'storyDesc', 'unitsCount', 'yearbuilteffective']
    df_b6 = pd.DataFrame(house['property'][0]['building']['summary'],index=[0])
    
    ### AVM
    #df_v1 ['scr', 'value', 'high', 'low', 'valueRange']
    df_v1 = pd.DataFrame(house['property'][0]['avm']['amount'],index=[0])
    
    #df_v2 ['avmpoorlow', 'avmpoorhigh', 'avmpoorscore', 'avmgoodlow', 'avmgoodhigh', 
    #      'avmgoodscore', 'avmexcellentlow', 'avmexcellenthigh', 'avmexcellentscore']
    df_v2 = pd.DataFrame(house['property'][0]['avm']['condition'],index=[0])
    
    #df_v3 ['taxamt', 'taxpersizeunit', 'taxyear']
    df_v3 = pd.DataFrame(house['property'][0]['assessment']['tax'],index=[0])
    
    # Build df for property 1
    prop_1 = pd.concat([df_1['fips'],df_2,df_3,df_4,df_5,df_6,df_7,df_9,df_b1,
                        df_b2,df_b3,df_b4,df_b5,df_b6,df_v1,df_v2,df_v3],axis=1)
    
    return prop_1

In [24]:
extract_details(call_api_all('2960','Inca','St','UNIT','501','Denver','CO'))

address1=2960%20Inca%20St%20UNIT%20501 &address2=Denver%2C%20CO


,fips,depth,frontage,lotsize1,lotsize2,pooltype,countrysecsubd,countyuse1,muncode,munname,...,prkgSize,archStyle,bldgsNum,bldgType,imprType,levels,mobileHomeInd,storyDesc,unitsCount,yearbuilteffective
0,08031,0,0,0.0,0,NONE,Denver County,104,DE,DENVER,...,0,ROW HOUSE,0,TYPE UNKNOWN,RESIDENTIAL,2,,TYPE UNKNOWN,0,2001


In [69]:
df = extract_details(call_api_all('2960','Inca','St','UNIT','501','Denver','CO'))

address1=2960%20Inca%20St%20UNIT%20501 &address2=Denver%2C%20CO


In [71]:
df.shape

(1, 90)